In [10]:
from dash import html, dcc, Dash
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc

import pandas as pd
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go

# Dashboard

In [11]:
app = Dash(external_stylesheets=[dbc.themes.BOOTSTRAP], title="2022 Wars Effect")

## Import Needed Data

In [12]:
df = pd.read_csv('data/population.csv')
df_all_years = pd.read_csv('data/all_years_population.csv')
lons_lats = pd.read_csv('data/countries_codes_and_coordinates.csv',  sep=',', engine='python')

### Data Preprocess

In [13]:
for col in lons_lats.columns:
    lons_lats[col] = lons_lats[col].str.replace('"', '').str.strip()

lons_lats['Latitude (average)'] = lons_lats['Latitude (average)'].astype(float)
lons_lats['Longitude (average)'] = lons_lats['Longitude (average)'].astype(float)

## Title

In [14]:
title = dbc.Container(
    [
        dbc.Row(
            [
                html.H1("2022 Wars Effect", style={"font-family": "serif", "text-align": "center", 'color': '#C10000'})
            ], className="mt-3")
    ]
)

## Asylum-seekers, Refugees, IDPs along the years

In [15]:
total_three = df_all_years.groupby('Year').agg({'Asylum-seekers': 'sum', "Refugees under UNHCR's mandate": 'sum', "IDPs of concern to UNHCR": 'sum'})
three_lines_plot = px.line(data_frame=total_three, y=['Asylum-seekers', "Refugees under UNHCR's mandate", "IDPs of concern to UNHCR"])

In [16]:
plot_3 = dbc.Row([
    dcc.Graph(id='line_plot', figure=three_lines_plot)
])

## IDPs

In [26]:
idps = df_all_years[df_all_years['Year'] > 2020].groupby('Country of origin').agg({'IDPs of concern to UNHCR': 'sum'}).sort_values('IDPs of concern to UNHCR', ascending=False)
idps = idps.reset_index()
idps= idps.iloc[:10]

idps['color_seq'] = '#609cd4'
# idps.loc[idps['Country of origin'] == 'Syrian Arab Rep.', 'color_seq'] = '#ec7c34'
idps.loc[idps['Country of origin'] == 'Syrian Arab Rep.', 'color_seq'] = '#d62728'

idb_plot = px.bar(data_frame=idps, x = 'Country of origin', y="IDPs of concern to UNHCR", title='IDPs in The Last Three Years', color = 'Country of origin', color_discrete_sequence=idps['color_seq'].values)
idb_plot.update_layout(title_text='IDPs In The Last Three Years', title_x=0.5);

In [27]:
plot_4 = dbc.Col([
    dcc.Graph(id='idp_plot', figure=idb_plot)
], className='col-6 p-0')

## Migrations Connections

In [28]:
iso_loc = lons_lats[['Alpha-3 code', 'Latitude (average)', 'Longitude (average)', 'Country']]

isos_original = df_all_years[['Country of origin (ISO)', 'Country of asylum (ISO)']]
isos_original = isos_original[isos_original['Country of origin (ISO)'] == 'SYR'].drop_duplicates()

df_cd = pd.merge(isos_original, iso_loc, how='left', left_on = 'Country of asylum (ISO)', right_on = 'Alpha-3 code')

In [29]:
mig_fig = go.Figure()

dest = zip(df_cd["Latitude (average)"], df_cd["Longitude (average)"])
source_lat_long = iso_loc[iso_loc['Alpha-3 code'] == 'SYR']
slat, slon =source_lat_long.iloc[0, 1:3]

## Loop thorugh each flight entry to add line between source and destination
for dlat, dlon in dest:
    mig_fig.add_trace(go.Scattergeo(
                        lat = [slat,dlat],
                        lon = [slon, dlon],
                        mode = 'lines',
                        line = dict(width = 1, color="red")
                        ))

## Loop thorugh each flight entry to plot source and destination as points.
mig_fig.add_trace(
    go.Scattergeo(
                lon =  df_cd["Longitude (average)"].values.tolist(),
                lat =  df_cd["Latitude (average)"].values.tolist(),
                hoverinfo = 'text',
                text = df_cd['Country'],
                mode = 'markers',
                marker = dict(size = 5, color = 'blue', opacity=0.6))
    )

## Update graph layout to improve graph styling.
mig_fig.update_layout(
                  title_text="Refuegees from Syria to other countries",
                  # height=500, 
                # width=900,
                  margin={"t":40,"b":0,"l":0, "r":0, "pad":0},
                  showlegend=False,
                  title_x=0.5
                  # geo= dict(showland = True, landcolor = 'white', countrycolor = 'grey', bgcolor="lightgrey")
                  )

mig_fig.update_geos(projection_type="natural earth");

In [30]:
plot_5 = dbc.Col([
    dcc.Graph(id='mig_plot', figure=mig_fig)
], className='col-6 p-0')

In [31]:
last_row = dbc.Row([
    plot_4,
    plot_5
])

## Tabs

In [32]:
analysis_content = dbc.Card(
    dbc.CardBody(
        [
            dbc.Container([
                #plot_1, 2,
                plot_3,
                last_row
            ])
        ]
    ),
    className="mt-3",
)

donation_content = dbc.Card(
    dbc.CardBody(
        [
            "bla bla"
        ]
    ),
    className="mt-3",
)

tabs = dbc.Tabs(
    [
        dbc.Tab(analysis_content, label="Analysis"),
        dbc.Tab(donation_content, label="Donation")
    ]
)

In [33]:
app.layout = html.Div(children=[
    title,
    html.Div(tabs, style={'margin': '30px'})
])

# Run Server

In [34]:
app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [13/Mar/2023 01:32:55] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2023 01:32:55] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2023 01:32:55] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2023 01:32:55] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2023 01:32:55] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
